In [ ]:
#This notebook imports Oneview in-situ data as a hyperspy stack

In [1]:
%matplotlib qt5

In [2]:
    import hyperspy.api as hs
    import numpy as np
    import os

In [3]:
def OneView_IS_import(folder_path):
    """
    Imports OneView_IS files.
    These files are saved as nested folders with the format:
    'Dataset/Hour_00/Minute_00/Second_00/*.dm4' etc.
    Function parameters:
    
    folder_path (string): the directory path of the Dataset folder
    ----------
    Retuns:
    imageStack_hs: Hypespy Signal2D lazy stack of the frames, ordered sequentially
                    The image axes parameters are copied over from first frame.
                    The thrid axis is defines as 't', scaled with the camera exposure time. 
                    OneView_IS captures the frames continuously. 
    """
    
    os.chdir(folder_path)
    
    filenames_sort = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".dm4") or file.endswith(".dm3"):
                filenames_sort.append(os.path.join(root, file))
                
    filenames_sort.sort()
    
    imageStack_hs = hs.load(filenames_sort, lazy = True, stack = True)
    
    imageStack_hs.axes_manager[0].name = 't'
    imageStack_hs.axes_manager[0].units = 'sec'
    imageStack_hs.axes_manager[0].scale = imageStack_hs.inav[0].metadata.Acquisition_instrument.TEM.Camera.exposure
    
    imageStack_hs.axes_manager['t'].index = 0 #For some reason the index is changed to 211! Resettting to 0 here
    
    return imageStack_hs

In [4]:
data = OneView_IS_import(r'Z:\Mohsen\Yasser Sessions\20180425-MD-CsPbBr3-Dose\4- 80kV\Dataset36')

In [5]:
data

<LazySignal2D, title: Second_00, dimensions: (470|1024, 1024)>

In [6]:
data.axes_manager

<Axes manager, axes: (470|1024, 1024)>
            Name |   size |  index |  offset |   scale |  units 
================ | ====== | ====== | ======= | ======= | ====== 
               t |    470 |      0 |       0 |     0.1 |    sec 
---------------- | ------ | ------ | ------- | ------- | ------ 
               x |   1024 |        |      -0 |   0.012 |   1/nm 
               y |   1024 |        |      -0 |   0.012 |   1/nm

In [9]:
data.save('Dataset36', extension = 'hdf5')